In [158]:
import pyproj
from  pyproj  import  CRS
from pyproj import Proj
import h5py
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm, colors
import glob
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset
import pandas as pd
import gzip
from collections import defaultdict

In [159]:
# 设置空间分辨率
resolution = 10000

# 将WGS 84坐标（4326）转化为等距圆柱投影（4088）
crs = CRS.from_epsg(4326)
crs = CRS.from_string("epsg:4326")
crs = CRS.from_proj4("+proj=latlon")
crs = CRS.from_user_input(4326)
crs2 = CRS.from_epsg(4088)
crs2 = CRS.from_string("epsg:4088")
crs2 = CRS.from_proj4("+proj=latlon")
crs2 = CRS.from_user_input(4088)

transformer = pyproj.Transformer.from_crs(crs,crs2)
transformer_back = pyproj.Transformer.from_crs(crs2,crs)

In [160]:
# 获取数组的长和宽
def get_nlat_nlon_npaeqd(resolution):
    nlat, nlon =40000000/ resolution,40000000/ resolution
    nlat = np.int(nlat)
    nlon = np.int(nlon)
    return nlat, nlon
    
    
def get_nlat_nlon_cyl(resolution):
    nlat, nlon =40000000/ resolution,20000000/ resolution
    nlat = np.int(nlat)
    nlon = np.int(nlon)
    return nlat, nlon

# 获取用来存放数据的grid数组，及其对应的grid_num数组（用来存放一个格子中有多少重合点），以及对应的初始time数组
def get_swh_grid(nlat, nlon):
    swh_grid = np.full(shape=(nlon,nlat), fill_value=np.nan)
    swh_grid_num = np.zeros((nlon,nlat))
    swh_grid_time = np.full(shape=(nlon,nlat), fill_value=np.nan)
    return swh_grid, swh_grid_num, swh_grid_time

# 填充每个点的数值，将有重复数据的点标记到swh_grid_num中,最后求平均
def coincident_point_mean(value_array, grid_array, grid_num_array, projlats, projlons):
    for i in range(len(value_array)):
        x = int((projlons[i] )/resolution)
        y = int(projlats[i]/resolution)
        if grid_num_array[x][y] == 0:
            grid_array[x][y] = value_array[i]
            grid_num_array[x][y] += 1
        else:
            grid_array[x][y] += value_array[i]
            grid_num_array[x][y] += 1
    grid_array = grid_array / grid_num_array
    return grid_array



In [210]:
# hy_dir_path = r'G:\remote_sensing_data\HY-2B\2020\07\20200701_20200711'
hy_dir_path_01 = r'G:\remote_sensing_data\HY-2B\2020\07\20200701_20200711\20200701'
hyfiles = glob.glob(hy_dir_path_01 + '\*.nc')
# hyfile = r'G:\remote_sensing_data\HY-2B\2020\07\20200711\H2B_OPER_GDR_2PC_0045_0116_20200711T001410_20200711T010412.nc'
hy_lon_array = np.array([])
hy_lat_array = np.array([])
hy_time_array = np.array([])
hy_value_array = np.array([])
hy_swhc_mask_array = np.array([])

for hyfile in hyfiles:
    with Dataset(hyfile, mode='r') as fh:
        lons = fh.variables['lon'][:]
        lats = fh.variables['lat'][:]
        swhc = fh.variables['swh_c'][:]
        time = fh.variables['time'][:]
        swhc_mask = swhc.mask
        hy_lon_array = np.append(hy_lon_array, lons)
        hy_lat_array = np.append(hy_lat_array, lats)
        hy_value_array = np.append(hy_value_array, swhc)
        hy_time_array = np.append(hy_time_array, time)
        hy_swhc_mask_array = np.append(hy_swhc_mask_array, swhc_mask)
    hy_value_array_masked = np.ma.array(hy_value_array, mask=hy_swhc_mask_array)
hy_df = pd.DataFrame([hy_lon_array, hy_lat_array, hy_time_array, hy_value_array_masked], index=['lon', 'lat', 'time', 'value'])

In [212]:
hy_projlats,hy_projlons = transformer.transform(hy_lat_array,hy_lon_array)
# 获取东西半球的nlat，nlon，is_grid_west,is_grid_num_west
hy_nlat, hy_nlon =get_nlat_nlon_cyl(resolution)
hy_grid, hy_num_grip, hy_time_grid = get_swh_grid(hy_nlat, hy_nlon)

In [213]:
hy_time_dict = {}
for i in range(len(hy_value_array_masked)):
    x = int((hy_projlats[i] )/resolution)
    y = int(hy_projlons[i]/resolution)
    if hy_num_grip[x][y] == 0:
        hy_grid[x][y] = hy_value_array_masked[i]
        hy_time_grid[x][y] = hy_time_array[i]
        hy_num_grip[x][y] = 1
        dict_name = str(x) + '+' + str(y)
        hy_time_dict[dict_name] = {hy_time_grid[x][y]: hy_grid[x][y]}
    else:
        dict_name = str(x) + '+' + str(y)
        hy_time_dict[dict_name][hy_time_array[i]] = hy_value_array_masked[i]


<ipython-input-213-5302c47a72fb>:6: UserWarning: Warning: converting a masked element to nan.
  hy_grid[x][y] = hy_value_array_masked[i]


KeyError: '-1128+-292'

In [207]:
# 删除没有重合点的子字典
for key in list(hy_time_dict.keys()):
    if len(hy_time_dict[key].items()) < 2 :
        del hy_time_dict[key]